In [2]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [3]:
import os
import numpy as np 
import csv
from glob import glob

In [6]:
import pandas as pd  
import matplotlib.pyplot as plt  
%matplotlib inline

from sklearn.model_selection import train_test_split  
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error

In [7]:
ROOT_DIR = os.path.abspath("../../")
DS_DIR = os.path.join(ROOT_DIR, "Datasets/awe")


### Model generation using Imagenet weights

In [13]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()

W0814 09:57:13.653600 140484881557312 deprecation_wrapper.py:119] From /home/sol/.virtualenvs/maskrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 09:57:13.673093 140484881557312 deprecation_wrapper.py:119] From /home/sol/.virtualenvs/maskrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 09:57:13.679313 140484881557312 deprecation_wrapper.py:119] From /home/sol/.virtualenvs/maskrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 09:57:13.703358 140484881557312 deprecation_wrapper.py:119] From /home/sol/.virtualenvs/maskrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please us

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Image Feature Extraction

In [79]:
def getVector():
    return np.array([1,2,3,5,6,7,8,9,10])


In [3]:
vgg16_feature_list = []
for (dirpath, u_dirnames, filenames) in os.walk(DS_DIR):
    dirnames = sorted(u_dirnames)
    for i in range(0,len(dirnames)):
        FILE_DIR = os.path.join(DS_DIR, str(dirnames[i]))
        files = glob(FILE_DIR+'/*.png') 
        for j in range (0,len(files)):
            #print(files[j])
            image_class = np.array([int(str(dirnames[i]))])
            img = image.load_img(files[j], target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)
            vgg16_feature = model.predict(img_data)
            image_vector = np.array(vgg16_feature)            
            #image_vector = getVector()                       
            vgg16_feature_np = np.concatenate((image_class, image_vector), axis=0)                                  
            vgg16_feature_list.append(vgg16_feature_np.flatten())            
    break

print(len(vgg16_feature_list))
print(len(vgg16_feature_np))
head = []
head.append('class')
for i in range(1, len(vgg16_feature_np)):
    head.append('v'+str(i))

head_np = []
head_np.append(head)
print(head_np)

NameError: name 'os' is not defined

### Build CSV file

In [115]:
with open('vgg16.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(head_np)
    writer.writerows(vgg16_feature_list)
csvFile.close()

### Get Distance metric

In [8]:
def chiSquared(a,b):
    res = np.sqrt(0.5*np.sum((a-b)**2/(a+b+1e-6)))
    return res

def chiSquared2(a,b):
    #res = np.sqrt(0.5*np.sum((a-b)**2/(a+b+1e-6)))
    return 0.5*np.sum((a-b)**2/(a+b+1e-6))

def getMetrics(csvfilename):
    data = pd.read_csv(csvfilename)
    X = data.drop('class', axis=1)  
    y = data['class']
    y_pred = np.zeros(data.shape[0])
    for i in range(len(X)):
        minimun = 1e+10;    
        for j in range (len(X)):
            res = chiSquared(X.values[i], X.values[j])
            if res < minimun and i != j:
                minimun = res
                y_pred[i] = y[j] 
                
    count = 0
    for i in range (len(y)):
        if y[i] == y_pred[i]:
            count = count + 1    
    msq = np.sqrt(mean_squared_error(y, y_pred))
    rank1 = count*100/len(y)
    res = 'MSR: '+ str(msq) + ' - Rank1: '+ str(rank1)    
    return res

In [15]:
print(getMetrics("calculated_features/awe-awet_bsif.csv"))

MSR: 30.585764662666193 - Rank1: 44.0


In [137]:
print(getMetrics("calculated_features/awe-awet_dsift.csv"))

MSR: 32.4314970361 - Rank1: 39.3


In [9]:
print(getMetrics("calculated_features/awe-awet_hog.csv"))

MSR: 29.080921580995334 - Rank1: 44.4


In [10]:
print(getMetrics("calculated_features/awe-awet_lbp.csv"))

MSR: 29.929567320627942 - Rank1: 40.5


In [11]:
print(getMetrics("calculated_features/awe-awet_lpq.csv"))

MSR: 30.871961388936725 - Rank1: 38.9


In [12]:
print(getMetrics("calculated_features/awe-awet_rilpq.csv"))

MSR: 31.63822055678859 - Rank1: 37.8


In [14]:
print(getMetrics("calculated_features/awe-awet_poem.csv"))

MSR: 29.151157781467273 - Rank1: 46.8


In [134]:
print(getMetrics("calculated_features/vgg16.csv"))

MSR: 57.3018324314 - Rank1: 1.0
